In [ ]:
def prepare_xgboost_model(
        self,
        X: pd.DataFrame,
        y: pd.Series,
        params_override: Optional[Dict] = None,
        fit_kwargs: Optional[Dict] = None,
        save_overwrite: bool = True,):
        """
        Train an XGBoost model using configuration from ConfigurationManager.get_xgboost_config()
        and save the fitted model to the configured path.

        Returns the fitted model instance.
        """
        # load config for xgboost
        cfg = ConfigurationManager().get_xgboost_config()
        model_cfg = cfg.get("params", {}).copy()
        train_cfg = cfg.get("train", {}) or {}

        # allow runtime overrides
        if params_override:
            model_cfg.update(params_override)

        fit_opts = train_cfg.get("fit_kwargs", {}) or {}
        if fit_kwargs:
            fit_opts.update(fit_kwargs)

        # choose classifier vs regressor by objective string
        objective = str(model_cfg.get("objective", "")).lower()
        if objective.startswith("reg:") or "reg" in objective or "squarederror" in objective:
            ModelClass = xgb.XGBRegressor
        else:
            ModelClass = xgb.XGBClassifier

        # instantiate and fit
        model = ModelClass(**model_cfg)
        model.fit(X, y, **fit_opts)

        # determine save location from config and persist
        model_path = Path(cfg.get("model_path", self.root_dir / "xgb_model.pkl"))
        model_path.parent.mkdir(parents=True, exist_ok=True)
        if model_path.exists() and not save_overwrite:
            raise FileExistsError(f"XGBoost model already exists at {model_path}")

        with open(model_path, "wb") as f:
            pickle.dump(model, f)

        try:
            logger.info(f"XGBoost model trained and saved to: {model_path}")
        except Exception:
            pass

        return model